En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [38]:
file_path = "../data/farmers-protest-tweets-2021-2-4.json"

# LATAM Challenge

Este proyecto se centra en la resolución de un desafío de ingeniería de datos que implica el procesamiento de un conjunto de datos de tweets relacionados con protestas agrícolas, según el nombre del archivo proporcionado. Se abordarán dos formas para dar solución a este reto, optimizando tanto el tiempo de ejecución como el uso de memoria. 

## Enfoques

### Enfoque Local

El enfoque local se implementará utilizando pandas, json, entre otras librerías. Este enfoque es adecuado para el procesamiento de datos en un entorno de desarrollo local o en nube pero con pocos datos, es fácil de implementar y depurar. Sin embargo, puede no ser adecuado para conjuntos de datos muy grandes debido a limitaciones de memoria y rendimiento.


### Enfoque Distribuido

El enfoque distribuido se implementará utilizando PySpark, que es un marco de computación distribuida que permite el procesamiento de grandes volúmenes de datos de manera eficiente. Aunque en este contexto los datos no son tan grandes y no se ejecutó en un clúster, la implementación de las funciones con PySpark se propone para demostrar cómo se podría escalar el procesamiento en entornos distribuidos. Tecnologías como Databricks, AWS Glue, Google Cloud Dataproc / BigQuery y Azure HDInsight permiten ejecutar PySpark en un clúster, ofreciendo ventajas significativas para grandes conjuntos de datos. Sin embargo, dado el tamaño relativamente pequeño de los datos en este caso, no se observarían muchas ventajas asi se ejecute en un clúster.



## Descripción del Problema

Fue proporcionado en el reto un archivo JSON que contiene aproximadamente 398 MB de datos de tweets. Se pide resolver los siguientes problemas:

1. **Top 10 Fechas con Más Tweets**: Identificar las 10 fechas con más tweets y mencionar el usuario que más publicaciones tiene por cada una de esas fechas.
2. **Top 10 Emojis Más Usados**: Identificar los 10 emojis más usados con su respectivo conteo.
3. **Top 10 Usuarios Más Mencionados**: Identificar los 10 usuarios más influyentes en función del conteo de menciones (@) que registra cada uno de ellos.

Cada problema se resolverá utilizando dos enfoques:
- **Optimización para el Tiempo de Ejecución**
- **Optimización para el Uso de Memoria**

### Consideraciones sobre Tiempo y Consumo de Memoria

Es importante destacar que tanto el tiempo de ejecución como el consumo de memoria dependen significativamente del hardware utilizado, así como de la metodología implementada. El rendimiento puede variar considerablemente entre diferentes configuraciones de hardware y enfoques de procesamiento de datos.

## Enfoque Local *

### Librerías

In [44]:
# Importando librerías necesarias
import pandas as pd
from typing import List, Tuple
import datetime
import json
from collections import defaultdict
import time
from memory_profiler import profile
from memory_profiler import memory_usage


# Configuración de pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

### Lectura de los datos

In [47]:
df = pd.read_json(file_path, lines=True)
df.head()

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,0,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24 09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,0,0,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'Kisanektamorcha', 'displayname'..."
2,https://twitter.com/parmarmaninder/status/1364...,2021-02-24 09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ...",[],[],0,0,0,0,1364506195453767680,pa,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",[https://youtu.be/-bUKumwq-J8],[https://t.co/wBPNdJdB0n],0,0,0,0,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'ReallySwara', 'displayname': 'S..."
4,https://twitter.com/KotiaPreet/status/13645061...,2021-02-24 09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre...",[],[],0,0,0,0,1364506144002088963,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None


### Descripción de las Columnas del DataFrame

1. **url**: La URL del tweet.
2. **date**: La fecha y hora en que se creó el tweet, en formato UTC.
3. **content**: El texto del tweet.
4. **renderedContent**: El contenido renderizado del tweet (puede ser igual a `content`).
5. **id**: El identificador único del tweet.
6. **user**: Informacion sobre usuario que publicó el tweet.
7. **outlinks**: Enlaces externos incluidos en el tweet.
8. **tcooutlinks**: Enlaces acortados de Twitter incluidos en el tweet.
9. **replyCount**: Número de respuestas al tweet.
10. **retweetCount**: Número de retweets del tweet.
11. **likeCount**: Número de "me gusta" que recibió el tweet.
12. **quoteCount**: Número de veces que el tweet fue citado.
13. **conversationId**: Identificador de la conversación a la que pertenece el tweet.
14. **lang**: El idioma del tweet, en formato de código BCP 47.
15. **source**: La fuente desde donde se publicó el tweet (por ejemplo, "Twitter Web Client").
16. **sourceUrl**: URL de la fuente desde donde se publicó el tweet.
17. **sourceLabel**: Etiqueta de la fuente desde donde se publicó el tweet.
18. **media**: Información sobre los medios incluidos en el tweet (imágenes, videos, etc.).
19. **retweetedTweet**: Información sobre el tweet original si este es un retweet.
20. **quotedTweet**: Información sobre el tweet citado.
21. **mentionedUsers**: Usuarios mencionados en el tweet.


In [49]:
# Ahora verifiquemos la cantidad de observaciones
print('El número de observaciones es', df.shape[0])

El número de observaciones es 117407


### <span style="color:red">Las Top 10 Fechas con Más Tweets y el Usuario Más Activo en Cada Fecha</span>


`q1_time:` Está optimizada para el tiempo de ejecución. Utiliza pandas para cargar y procesar todo el archivo en memoria, lo que permite realizar operaciones vectorizadas rápidas y eficientes en términos de tiempo, pero puede consumir más memoria, una vez cargado el df en memoria se podria hacer todas las operaciones necesarias de froma muy rapida.

`q1_memory:` Está optimizada para el uso de memoria. Utiliza técnicas que requieren menos memoria, como procesar el archivo línea por línea en lugar de cargarlo todo en memoria a la vez. Esto puede ser más lento debido a la falta de operaciones vectorizadas, pero reduce el uso de memoria.

In [58]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str, int]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de las
    10 fechas con más tweets y el nombre del usuario que más publicó en cada una de esas fechas.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[datetime.date, str]]: Una lista de tuplas donde cada tupla contiene una fecha (datetime.date)
    y el nombre del usuario que más publicó en esa fecha.
    """
    # Cargar el archivo JSON en un DataFrame de pandas
    df = pd.read_json(file_path, lines=True)
    
    # Convertir la columna 'date' a datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Obtener las 10 fechas con más tweets
    top_dates = df['date'].dt.date.value_counts().nlargest(10)
    
    # Crear una lista de tuplas con las fechas y el nombre de usuario más activo en cada fecha
    result = [(date, df[df['date'].dt.date == date]['user'].apply(lambda x: x['username']).value_counts().idxmax()) for date in top_dates.index]
    
    return result

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_time = memory_usage((q1_time, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q1_time: {end_time - start_time} segundos")
print(f"Uso de memoria de q1_time: {max(mem_usage_time) - min(mem_usage_time)} MiB")
print(q1_time(file_path))

Tiempo de ejecución de q1_time: 6.856872081756592 segundos
Uso de memoria de q1_time: 1794.34375 MiB
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [59]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de las
    10 fechas con más tweets y el nombre del usuario que más publicó en cada una de esas fechas,
    optimizando para el uso de memoria.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[datetime.date, str]]: Una lista de tuplas donde cada tupla contiene una fecha (datetime.date)
    y el nombre del usuario que más publicó en esa fecha.
    """
    # Estructura para contar los tweets por fecha y por usuario
    date_counts = defaultdict(lambda: defaultdict(int))

    # Procesar el archivo línea por línea para minimizar el uso de memoria
    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line) # Convert to Python Dict
            date = datetime.date.fromisoformat(tweet['date'][:10])
            user = tweet['user']['username']
            date_counts[date][user] += 1 # Incrementar para la fecha y usuario correspondiente

    # Obtener las 10 fechas con más tweets
    top_dates = sorted(date_counts.items(), key=lambda x: sum(x[1].values()), reverse=True)[:10] #x[1] conteos por usuario

    # Crear una lista de tuplas con las fechas y el nombre de usuario más activo en cada fecha
    result = [(date, max(users.items(), key=lambda x: x[1])[0]) for date, users in top_dates] #sum(users.values())

    return result

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_memory = memory_usage((q1_memory, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q1_memory: {end_time - start_time} segundos")
print(f"Uso de memoria de q1_memory: {max(mem_usage_memory) - min(mem_usage_memory)} MiB")
print(q1_memory(file_path))


Tiempo de ejecución de q1_memory: 2.625170946121216 segundos
Uso de memoria de q1_memory: 1.0 MiB
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


### Resultado

`q1_memory` es más eficiente tanto en términos de tiempo de ejecución como de uso de memoria para este conjunto de datos específico. Aunque pandas es generalmente muy eficiente para operaciones vectorizadas, el tiempo y la memoria necesarios para cargar un archivo JSON grande pueden no justificar su uso en todos los casos. Si el archivo JSON es pequeño o de tamaño moderado, la carga completa en un DataFrame de pandas puede ser lo suficientemente rápida y eficiente, aprovechando las operaciones vectorizadas de pandas para procesar los datos rápidamente. Si el archivo JSON está estructurado de una manera más plana y simple, pandas puede cargar y procesar los datos más rápidamente.

### <span style="color:red">Los Top 10 Emojis Más Usados con su Respectivo Conteo</span>


`q2_time:` Está optimizada para el tiempo de ejecución. Utiliza pandas para cargar y procesar todo el archivo en memoria, lo que permite realizar operaciones vectorizadas rápidas y eficientes en términos de tiempo, pero puede consumir más memoria, una vez cargado el df en memoria se podria hacer todas las operaciones necesarias de froma muy rapida.

`q2_memory:` Está optimizada para el uso de memoria. Utiliza técnicas que requieren menos memoria, como procesar el archivo línea por línea en lugar de cargarlo todo en memoria a la vez. Esto puede ser más lento debido a la falta de operaciones vectorizadas, pero reduce el uso de memoria.

In [ ]:
import emoji

def extract_emojis_from_text(text):
    return [c for c in text if c in emoji.EMOJI_DATA]

In [ ]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de los
    10 emojis más usados con su respectivo conteo.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[str, int]]: Una lista de tuplas donde cada tupla contiene un emoji (str) y su conteo (int).
    """
    # Cargar el archivo JSON en un DataFrame de pandas
    df = pd.read_json(file_path, lines=True)
    
    # Extraer todos los emojis de los tweets
    df['emojis'] = df['content'].apply(extract_emojis_from_text)
    
    # Contar la frecuencia de cada emoji
    all_emojis = df['emojis'].explode() # serie de emojis
    top_emojis = all_emojis.value_counts().nlargest(10)
    
    # Crear una lista de tuplas con los emojis y su conteo
    result = list(top_emojis.items())
    
    #total_emojis = all_emojis.count()
    #result = [(emoji, count, (count / total_emojis) * 100) for emoji, count in top_emojis.items()]
    return result

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_time = memory_usage((q2_time, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q2_time: {end_time - start_time} segundos")
print(f"Uso de memoria de q2_time: {max(mem_usage_time) - min(mem_usage_time)} MiB")
print(q2_time(file_path))

In [ ]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de los
    10 emojis más usados con su respectivo conteo, optimizando para el uso de memoria.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[str, int]]: Una lista de tuplas donde cada tupla contiene un emoji (str) y su conteo (int).
    """
    # Estructura para contar los emojis
    emoji_counts = defaultdict(int)
    #total_emojis = 0

    # Procesar el archivo línea por línea para minimizar el uso de memoria
    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line)
            content = tweet['content']
            emojis_in_tweet = extract_emojis_from_text(content)
            for em in emojis_in_tweet:
                emoji_counts[em] += 1
                #total_emojis += 1

    # Obtener los 10 emojis más usados
    top_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    #result = [(emoji, count, (count / total_emojis) * 100) for emoji, count in top_emojis]

    return top_emojis

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_memory = memory_usage((q2_memory, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q2_memory: {end_time - start_time} segundos")
print(f"Uso de memoria de q2_memory: {max(mem_usage_memory) - min(mem_usage_memory)} MiB")
print(q2_memory(file_path))


### Resultado

`q2_memory` es más eficiente tanto en términos de tiempo de ejecución como de uso de memoria para este conjunto de datos específico. Aunque pandas es generalmente muy eficiente para operaciones vectorizadas, el tiempo y la memoria necesarios para cargar un archivo JSON grande pueden no justificar su uso en todos los casos. La elección del método de procesamiento depende del tamaño y estructura del archivo de datos y de los recursos del sistema disponibles. En situaciones donde se dispone de suficiente memoria y el archivo de datos no es excesivamente grande, el enfoque basado en pandas puede ser adecuado y eficiente. 

### <span style="color:red">Top 10 Histórico de Usuarios Más Influyentes en Función del Conteo de las Menciones (@)</span>

`q3_time:` Está optimizada para el tiempo de ejecución. 
`q3_memory:` Está optimizada para el uso de memoria.

In [ ]:
import re
def extract_mentions(s):
    return re.findall(r'@\w+', s)

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de los
    10 usuarios más mencionados con su respectivo conteo.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[str, int]]: Una lista de tuplas donde cada tupla contiene un usuario (str) y su conteo de menciones (int).
    """
    # Cargar el archivo JSON en un DataFrame de pandas
    df = pd.read_json(file_path, lines=True)
    
    # Extraer todas las menciones de los tweets
    df['mentions'] = df['content'].apply(extract_mentions)
    
    # Contar la frecuencia de cada mención
    all_mentions = df['mentions'].explode()
    top_mentions = all_mentions.value_counts().nlargest(10)
    
    # Crear una lista de tuplas con los usuarios y su conteo de menciones
    result = list(top_mentions.items())
    
    return result

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_time = memory_usage((q3_time, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q3_time: {end_time - start_time} segundos")
print(f"Uso de memoria de q3_time: {max(mem_usage_time) - min(mem_usage_time)} MiB")
print(q3_time(file_path))


In [ ]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de los
    10 usuarios más mencionados con su respectivo conteo, optimizando para el uso de memoria.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[str, int]]: Una lista de tuplas donde cada tupla contiene un usuario (str) y su conteo de menciones (int).
    """
    # Estructura para contar las menciones
    mention_counts = defaultdict(int)

    # Procesar el archivo línea por línea para minimizar el uso de memoria
    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line)
            content = tweet['content']
            mentions_in_tweet = extract_mentions(content)
            for mention in mentions_in_tweet:
                mention_counts[mention] += 1

    # Obtener los 10 usuarios más mencionados
    top_mentions = sorted(mention_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_mentions

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_memory = memory_usage((q3_memory, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q3_memory: {end_time - start_time} segundos")
print(f"Uso de memoria de q3_memory: {max(mem_usage_memory) - min(mem_usage_memory)} MiB")
print(q3_memory(file_path))


### Resultado

`q3_memory` es más eficiente tanto en términos de tiempo de ejecución como de uso de memoria para este conjunto de datos específico. Aunque pandas es generalmente muy eficiente para operaciones vectorizadas, el tiempo y la memoria necesarios para cargar un archivo JSON grande pueden no justificar su uso en todos los casos. La elección del método de procesamiento depende del tamaño y estructura del archivo de datos y de los recursos del sistema disponibles. En situaciones donde se dispone de suficiente memoria y el archivo de datos no es excesivamente grande, el enfoque basado en pandas puede ser adecuado y eficiente. Sin embargo, para archivos de gran tamaño o sistemas con limitaciones de memoria, procesar el archivo línea por línea es una estrategia más robusta y eficiente. Evaluar estas variables es crucial para seleccionar la estrategia de procesamiento más adecuada para cada caso particular.

## Enfoque Distribuido en la Nube *

### <span style="color:red">Las Top 10 Fechas con Más Tweets y el Usuario Más Activo en Cada Fecha</span>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, to_date,split, udf
from pyspark.sql.types import ArrayType, StringType
from collections import defaultdict
import json
import datetime
from typing import List, Tuple
import time
from memory_profiler import memory_usage

# Inicializar Spark
spark = SparkSession.builder \
    .appName("Twitter Mentions Analysis") \
    .getOrCreate()


def q1_time_spark(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de las
    10 fechas con más tweets y el nombre del usuario que más publicó en cada una de esas fechas.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[datetime.date, str]]: Una lista de tuplas donde cada tupla contiene una fecha (datetime.date)
    y el nombre del usuario que más publicó en esa fecha.
    """
    # Cargar el archivo JSON en un DataFrame de Spark
    df = spark.read.json(file_path)

    # Convertir la columna 'date' a date
    df = df.withColumn('date', to_date(col('date')))
    
    # Obtener las 10 fechas con más tweets
    top_dates = df.groupBy('date').count().orderBy('count', ascending=False).limit(10).collect()
    
    # Crear una lista de tuplas con las fechas y el nombre de usuario más activo en cada fecha
    result = []
    for row in top_dates:
        date = row['date']
        top_user = df.filter(col('date') == date).groupBy('user.username').count().orderBy('count', ascending=False).first()['username']
        result.append((date, top_user))
    
    return result

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_time = memory_usage((q1_time_spark, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q1_time_spark: {end_time - start_time} segundos")
print(f"Uso de memoria de q1_time_spark: {max(mem_usage_time) - min(mem_usage_time)} MiB")
print(q1_time_spark(file_path))

### <span style="color:red">Los Top 10 Emojis Más Usados con su Respectivo Conteo</span>

In [ ]:
import emoji
def extract_emojis_from_text(text):
    return ''.join([c for c in text if c in emoji.EMOJI_DATA])

def q2_time_spark(file_path: str) -> List[Tuple[str, int]]:
    """
    Esta función toma la ruta de un archivo JSON que contiene tweets y devuelve una lista de los
    10 emojis más usados con su respectivo conteo.

    Parámetros:
    file_path (str): La ruta al archivo JSON que contiene los tweets.

    Retorna:
    List[Tuple[str, int]]: Una lista de tuplas donde cada tupla contiene un emoji (str) y su conteo (int).
    """
    # Cargar el archivo JSON en un DataFrame de Spark
    df = spark.read.json(file_path)
    
    # Extraer todos los emojis de los tweets
    extract_emojis_udf = spark.udf.register("extract_emojis_udf", extract_emojis_from_text)
    df = df.withColumn('emojis', extract_emojis_udf(col('content')))
    
    # Explode la columna de emojis
    df = df.withColumn('emoji', explode(split(col('emojis'), '')))
    
    # Contar la frecuencia de cada emoji
    emoji_counts = df.groupBy('emoji').count().orderBy('count', ascending=False).limit(10)
    
    # Crear una lista de tuplas con los emojis y su conteo
    result = [(row['emoji'], row['count']) for row in emoji_counts.collect()]
    
    return result

# Medir tiempo de ejecución
start_time = time.time()
mem_usage_time = memory_usage((q2_time_spark, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q2_time_spark: {end_time - start_time} segundos")
print(f"Uso de memoria de q2_time_spark: {max(mem_usage_time) - min(mem_usage_time)} MiB")
print(q2_time_spark(file_path))


### <span style="color:red">Top 10 Histórico de Usuarios Más Influyentes en Función del Conteo de las Menciones (@)</span>

In [ ]:
# función extract_mentions
def extract_mentions(s):
    return re.findall(r'@\w+', s)

# extract_mentions como UDF
extract_mentions_udf = udf(extract_mentions, ArrayType(StringType()))

def q3_time_spark(file_path: str) -> List[Tuple[str, int]]:
    df = spark.read.json(file_path)
    df = df.withColumn('mentions', explode(extract_mentions_udf(col('content'))))
    mention_counts = df.groupBy('mentions').count().orderBy('count', ascending=False).limit(10)
    result = [(row['mentions'], row['count']) for row in mention_counts.collect()]
    return result


# Medir tiempo de ejecución
start_time = time.time()
mem_usage_time = memory_usage((q3_time_spark, (file_path,)), interval=0.1, timeout=None)
end_time = time.time()

print(f"Tiempo de ejecución de q3_time_spark: {end_time - start_time} segundos")
print(f"Uso de memoria de q3_time_spark: {max(mem_usage_time) - min(mem_usage_time)} MiB")
print(q3_time_spark(file_path))


### Ventajas de Usar un Enfoque Distribuido

El uso de un enfoque distribuido para el procesamiento de datos tiene varias ventajas significativas, especialmente cuando se trabaja con grandes volúmenes de datos. Plataformas como Databricks, AWS (con AWS Glue), Azure (con Azure Synapse Analytics), y GCP (con Google BigQuery) ofrecen servicios robustos que facilitan la implementación y gestión de clústeres de computación distribuida.

1. **Escalabilidad**: Un enfoque distribuido permite escalar horizontalmente añadiendo más nodos al clúster. Esto significa que a medida que el volumen de datos crece, se pueden agregar más recursos para manejar la carga adicional, asegurando un rendimiento consistente.

2. **Eficiencia de Procesamiento**: Tecnologías como Apache Spark permiten el procesamiento paralelo de datos. Esto reduce significativamente el tiempo de ejecución para tareas que involucran grandes conjuntos de datos, ya que múltiples nodos pueden trabajar simultáneamente en diferentes partes del conjunto de datos.

3. **Tolerancia a Fallos**: Los sistemas distribuidos están diseñados para ser resilientes a fallos. Si un nodo falla, las tareas se pueden reasignar a otros nodos, asegurando que el procesamiento continúe sin interrupciones significativas.

4. **Flexibilidad y Facilidad de Uso**: Plataformas como Databricks proporcionan entornos integrados que facilitan la configuración y el uso de clústeres distribuidos. Además, ofrecen integraciones con una variedad de servicios en la nube, almacenamiento de datos, y herramientas de análisis, proporcionando un entorno de trabajo flexible y eficiente.

Sin embargo, es importante considerar que para conjuntos de datos pequeños, como el ejemplo de 398 MB utilizado en este ejercicio, las ventajas de un enfoque distribuido pueden no ser tan evidentes. La sobrecarga asociada con la configuración y gestión de clústeres puede superar los beneficios del procesamiento paralelo, resultando en tiempos de ejecución comparables o incluso mayores que los de un procesamiento local. No obstante, un enfoque distribuido se vuelve altamente beneficioso y necesario cuando se espera un crecimiento significativo en el volumen de datos o se requiere un procesamiento robusto y eficiente en un entorno de producción.

### Consideraciones de Escalabilidad

Para conjuntos de datos más grandes, el uso de Spark en un clúster distribuido proporciona una solución escalable y eficiente. A medida que el tamaño de los datos crece, el enfoque distribuido permite manejar la carga adicional sin degradar el rendimiento, algo que sería difícil de lograr con un enfoque local. Por lo tanto, para aplicaciones que implican grandes volúmenes de datos y requieren un rendimiento consistente, plataformas distribuidas como Databricks, AWS Glue, Azure Synapse Analytics, y Google BigQuery son las mejores opciones.

